Импортируйте нужные библиотеки
===
Мы уже много работали с  pandas, numpy и различными инструментами визуализации. Поэтому предлагаем вам самим импортировать те библиотеки, которые вам нужны для выполнения домашнего задания 

Временной ряд 
=== 
В папке datasets у нас находится множество различных временных рядов, которые состоят из двух колонок - времени и значения. Можно взять любые данные на выбор, кроме того, который мы разобрали на лекции (shampoo_sales). 

Задание 
==

* Сделайте визуализацию ряда
* Разбейте его на компоненты
* Проверьте автокорреляцию
* Примените одну или несколько трансформаций, разобранных на лекции